In [1]:
from pandas import read_csv as read
import pandas as pd
import numpy as np
import seaborn as sns; sns.set(color_codes=True)
import matplotlib.pyplot as plt

## Данные до исследования признаков

In [23]:
path = "filtData.csv"
data = read(path, delimiter=",")

data.drop('isFlaggedFraud', axis=1, inplace=True)
data.drop('nameOrig', axis=1, inplace=True)
data.drop('nameDest', axis=1, inplace=True)

'''
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
data['type'] = le.fit_transform(data['type'])
'''

data = pd.get_dummies(data)
data.head()

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,type_CASH_IN,type_CASH_OUT,type_DEBIT,type_PAYMENT,type_TRANSFER
0,1,9839.64,170136.00,160296.36,0.0,0.0,0,0,0,0,1,0
1,1,181.00,181.00,0.00,0.0,0.0,1,0,0,0,0,1
2,1,181.00,181.00,0.00,21182.0,0.0,1,0,1,0,0,0
3,1,9478.39,116494.00,107015.61,0.0,0.0,0,0,0,0,1,0
4,1,3454.08,9031.96,5577.88,0.0,0.0,0,0,0,0,1,0


In [24]:
data.corr()

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,type_CASH_IN,type_CASH_OUT,type_DEBIT,type_PAYMENT,type_TRANSFER
step,1.000000,0.097336,0.017761,-0.014001,0.014156,0.022648,0.214943,-0.025025,0.004669,-0.001878,-0.037757,0.080511
amount,0.097336,1.000000,0.205667,0.016601,0.195613,0.370067,0.346157,-0.054279,0.023805,-0.021563,-0.181568,0.304017
oldbalanceOrg,0.017761,0.205667,1.000000,0.966860,0.054735,0.052335,0.070277,0.464094,-0.180977,-0.022039,-0.190083,-0.034110
newbalanceOrig,-0.014001,0.016601,0.966860,1.000000,0.064145,0.036088,-0.060647,0.520536,-0.207895,-0.020425,-0.176750,-0.085203
oldbalanceDest,0.014156,0.195613,0.054735,0.064145,1.000000,0.967165,-0.041317,0.074350,0.082898,0.008319,-0.202006,0.073293
newbalanceDest,0.022648,0.370067,0.052335,0.036088,0.967165,1.000000,0.002590,0.029921,0.103833,0.005295,-0.211969,0.113545
isFraud,0.214943,0.346157,0.070277,-0.060647,-0.041317,0.002590,1.000000,-0.142891,0.082316,-0.021918,-0.189924,0.340712
type_CASH_IN,-0.025025,-0.054279,0.464094,0.520536,0.074350,0.029921,-0.142891,1.000000,-0.382152,-0.039765,-0.344581,-0.182274
type_CASH_OUT,0.004669,0.023805,-0.180977,-0.207895,0.082898,0.103833,0.082316,-0.382152,1.000000,-0.058617,-0.507939,-0.268685
type_DEBIT,-0.001878,-0.021563,-0.022039,-0.020425,0.008319,0.005295,-0.021918,-0.039765,-0.058617,1.000000,-0.052854,-0.027958


In [5]:
X = data.loc[:, data.columns != 'isFraud'].values
y = data.loc[:, 'isFraud'].values

y=y.astype('int') # не совсем понял, почему https://stackoverflow.com/questions/45346550/valueerror-unknown-label-type-unknown

from sklearn.model_selection import train_test_split as train

X_train, X_test, y_train, y_test = train(X, y, shuffle=False)

from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

model = RandomForestClassifier(n_estimators=110, n_jobs=-1)

model.fit(X_train, y_train)
expected = y_test
predicted = model.predict(X_test)

pobas = model.predict_proba(X_test)
preds = pobas[:, 1]

report = metrics.classification_report(expected, predicted)
conf_matrix = metrics.confusion_matrix(expected, predicted)

print(type(model).__name__)
print(report)
print(conf_matrix)
##print(metrics.roc_auc_score(expected, preds))

RandomForestClassifier
              precision    recall  f1-score   support

           0       0.99      1.00      0.99     23790
           1       0.99      0.93      0.96      4334

   micro avg       0.99      0.99      0.99     28124
   macro avg       0.99      0.97      0.98     28124
weighted avg       0.99      0.99      0.99     28124

[[23760    30]
 [  290  4044]]


## Данные после исследования признаков

In [20]:
import numpy as np
from pandas import read_csv as read
import pandas as pd
import seaborn as sns; sns.set(color_codes=True)
import matplotlib.pyplot as plt

path = "newFilt.csv"
data = read(path, delimiter=",")

data.drop('isFlaggedFraud', axis=1, inplace=True)
data.drop('nameOrig', axis=1, inplace=True)
data.drop('nameDest', axis=1, inplace=True)

data = pd.get_dummies(data)
data.head()

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,hour,newSender,newReceiver,merchant,fraudsEarly,LTS,LTR,IZoB,type_CASH_IN,type_CASH_OUT,type_DEBIT,type_PAYMENT,type_TRANSFER
0,1,9839.64,170136.00,160296.36,0.0,0.0,0,1,1,1,1,0,0,0,0,0,0,0,1,0
1,1,181.00,181.00,0.00,0.0,0.0,1,1,1,1,0,0,0,0,1,0,0,0,0,1
2,1,181.00,181.00,0.00,21182.0,0.0,1,1,1,1,0,0,0,0,1,0,1,0,0,0
3,1,9478.39,116494.00,107015.61,0.0,0.0,0,1,1,1,1,0,0,0,0,0,0,0,1,0
4,1,3454.08,9031.96,5577.88,0.0,0.0,0,1,1,1,1,0,0,0,0,0,0,0,1,0


In [21]:
data.corr()

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,hour,newSender,newReceiver,merchant,fraudsEarly,LTS,LTR,IZoB,type_CASH_IN,type_CASH_OUT,type_DEBIT,type_PAYMENT,type_TRANSFER
step,1.000000e+00,0.097336,0.017761,-0.014001,0.014156,0.022648,0.214943,-0.000397,-0.002345,-0.003239,-0.037757,-9.721728e-08,0.005863,0.125916,0.051057,-0.025025,0.004669,-0.001878,-0.037757,0.080511
amount,9.733587e-02,1.000000,0.205667,0.016601,0.195613,0.370067,0.346157,-0.081528,0.001350,-0.026341,-0.181568,2.111068e-03,-0.001002,0.033480,0.119121,-0.054279,0.023805,-0.021563,-0.181568,0.304017
oldbalanceOrg,1.776095e-02,0.205667,1.000000,0.966860,0.054735,0.052335,0.070277,-0.012752,0.001086,-0.053759,-0.190083,8.077165e-03,-0.001170,0.036172,-0.299378,0.464094,-0.180977,-0.022039,-0.190083,-0.034110
newbalanceOrig,-1.400124e-02,0.016601,0.966860,1.000000,0.064145,0.036088,-0.060647,0.012968,0.000825,-0.061909,-0.176750,9.924072e-03,-0.000938,0.039156,-0.342819,0.520536,-0.207895,-0.020425,-0.176750,-0.085203
oldbalanceDest,1.415617e-02,0.195613,0.054735,0.064145,1.000000,0.967165,-0.041317,0.007348,0.000838,-0.177631,-0.202006,4.372083e-02,-0.001316,0.173376,0.030357,0.074350,0.082898,0.008319,-0.202006,0.073293
newbalanceDest,2.264750e-02,0.370067,0.052335,0.036088,0.967165,1.000000,0.002590,-0.004981,0.001094,-0.168995,-0.211969,4.050082e-02,-0.001468,0.166395,0.073256,0.029921,0.103833,0.005295,-0.211969,0.113545
isFraud,2.149431e-01,0.346157,0.070277,-0.060647,-0.041317,0.002590,1.000000,-0.212034,0.001871,0.035525,-0.189924,-3.834196e-04,-0.001478,-0.002724,0.220020,-0.142891,0.082316,-0.021918,-0.189924,0.340712
hour,-3.970274e-04,-0.081528,-0.012752,0.012968,0.007348,-0.004981,-0.212034,1.000000,-0.003358,-0.007349,0.053131,-2.751391e-03,0.001089,0.002095,-0.062185,0.022327,-0.019689,-0.004976,0.053131,-0.075162
newSender,-2.345227e-03,0.001350,0.001086,0.000825,0.000838,0.001094,0.001871,-0.003358,1.000000,0.003078,0.004512,4.938944e-04,-0.789905,-0.001643,0.005377,-0.003201,-0.000551,0.000521,0.004512,-0.001815
newReceiver,-3.238786e-03,-0.026341,-0.053759,-0.061909,-0.177631,-0.168995,0.035525,-0.007349,0.003078,1.000000,0.195419,-2.560981e-01,-0.000518,-0.728641,-0.018265,-0.079805,-0.101021,-0.008383,0.195419,-0.029270


In [7]:
X = data.loc[:, data.columns != 'isFraud'].values
y = data.loc[:, 'isFraud'].values

y=y.astype('int') # не совсем понял, почему https://stackoverflow.com/questions/45346550/valueerror-unknown-label-type-unknown

from sklearn.model_selection import train_test_split as train

X_train, X_test, y_train, y_test = train(X, y, shuffle=False)

from sklearn import metrics

### Рандомный лес

In [8]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=110, n_jobs=-1)

model.fit(X_train, y_train)
expected = y_test
predicted = model.predict(X_test)

pobas = model.predict_proba(X_test)
preds = pobas[:, 1]

report = metrics.classification_report(expected, predicted)
conf_matrix = metrics.confusion_matrix(expected, predicted)

print(type(model).__name__)
print(report)
print(conf_matrix)
print(metrics.roc_auc_score(expected, preds))

RandomForestClassifier
              precision    recall  f1-score   support

           0       0.98      1.00      0.99     23790
           1       0.99      0.91      0.95      4334

   micro avg       0.99      0.99      0.99     28124
   macro avg       0.99      0.95      0.97     28124
weighted avg       0.99      0.99      0.98     28124

[[23769    21]
 [  398  3936]]
0.9995515094874335


### Решающее дерево

In [9]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier()

model.fit(X_train, y_train)
expected = y_test
predicted = model.predict(X_test)

pobas = model.predict_proba(X_test)
preds = pobas[:, 1]

report = metrics.classification_report(expected, predicted)
conf_matrix = metrics.confusion_matrix(expected, predicted)

print(type(model).__name__)
print(report)
print(conf_matrix)
print(metrics.roc_auc_score(expected, preds))

DecisionTreeClassifier
              precision    recall  f1-score   support

           0       0.99      1.00      0.99     23790
           1       0.99      0.95      0.97      4334

   micro avg       0.99      0.99      0.99     28124
   macro avg       0.99      0.97      0.98     28124
weighted avg       0.99      0.99      0.99     28124

[[23744    46]
 [  224  4110]]
0.9731910291034865


### К соседей

In [10]:
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier(n_neighbors=20)

model.fit(X_train, y_train)
expected = y_test
predicted = model.predict(X_test)

pobas = model.predict_proba(X_test)
preds = pobas[:, 1]

report = metrics.classification_report(expected, predicted)
conf_matrix = metrics.confusion_matrix(expected, predicted)

print(type(model).__name__)
print(report)
print(conf_matrix)
print(metrics.roc_auc_score(expected, preds))

KNeighborsClassifier
              precision    recall  f1-score   support

           0       0.96      1.00      0.98     23790
           1       0.97      0.79      0.87      4334

   micro avg       0.96      0.96      0.96     28124
   macro avg       0.97      0.90      0.93     28124
weighted avg       0.97      0.96      0.96     28124

[[23696    94]
 [  892  3442]]
0.9830973574149908


### ГаусианНБ

In [11]:
from sklearn.naive_bayes import GaussianNB

model = GaussianNB()

model.fit(X_train, y_train)
expected = y_test
predicted = model.predict(X_test)

pobas = model.predict_proba(X_test)
preds = pobas[:, 1]

report = metrics.classification_report(expected, predicted)
conf_matrix = metrics.confusion_matrix(expected, predicted)

print(type(model).__name__)
print(report)
print(conf_matrix)
print(metrics.roc_auc_score(expected, preds))

GaussianNB
              precision    recall  f1-score   support

           0       0.88      0.99      0.93     23790
           1       0.87      0.23      0.36      4334

   micro avg       0.88      0.88      0.88     28124
   macro avg       0.87      0.61      0.64     28124
weighted avg       0.87      0.88      0.84     28124

[[23645   145]
 [ 3357   977]]
0.6888099473686558


### Градиентный  бустинг

In [12]:
from sklearn.ensemble import GradientBoostingClassifier

model = GradientBoostingClassifier(max_depth=4)

model.fit(X_train, y_train)
expected = y_test
predicted = model.predict(X_test)

pobas = model.predict_proba(X_test)
preds = pobas[:, 1]

report = metrics.classification_report(expected, predicted)
conf_matrix = metrics.confusion_matrix(expected, predicted)

print(type(model).__name__)
print(report)
print(conf_matrix)
print(metrics.roc_auc_score(expected, preds))

GradientBoostingClassifier
              precision    recall  f1-score   support

           0       0.98      1.00      0.99     23790
           1       0.99      0.91      0.95      4334

   micro avg       0.99      0.99      0.99     28124
   macro avg       0.99      0.96      0.97     28124
weighted avg       0.99      0.99      0.99     28124

[[23765    25]
 [  374  3960]]
0.9994666743480923
